# Assignment 7: Neural Networks and Backpropagation
In this assignment, you will be asked to write your own code to implement the learning process of a simple neural network. We will use a simple version of [MNIST dataset](http://yann.lecun.com/exdb/mnist/), which we have introduced in Assignment 5. To make the problem simpler, we only take images with label '8' and '9', which gives us a binary classification problem. Then we subsample the dataset and reduce the dimension of each image using average pooling. The following code loads the dataset and prints its dimension.

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
#Load data
import scipy.io as sio
a = sio.loadmat('mnist_binary.mat')
X_trn = a['X_trn']
X_tst = a['X_tst']
Y_trn = a['Y_trn'][0]
Y_tst = a['Y_tst'][0]
print(X_trn.shape)
print(X_tst.shape)
print(Y_trn.shape)
print(Y_tst.shape)


(196, 964)
(196, 414)
(964,)
(414,)


## Requirements
1. You are not allowed to use any machine learning libraries which have neural networks implemented.

2. Notice here most of the problems you have will be regarding the dimensions of variables. In each skeleton function we provide, we have one assert line to help you verify whether you write your code correctly. Passing the assert line doesn't mean your code is correct. But it is a necessary condition.

3. You don't need to strictly follow the skeleton we provide. As long as you answer the problems correctly, you can write in any style you prefer.

## Parameters
Let's first implement a simple neural network with one hidden layer and one output layer. The hidden layer only has $n_h$ neurons. We assume the output layer has two neurons. Hence you will have 4 parameters to describe the neural network: 

1. $W_1$, a $n_h$ by 196 (number of features) matrix, which is the weight matrix between features and the hidder layer.
2. $b_1$, a scalar, which is the offset for the first layer.
3. $W_2$, a 2 by $n_h$ matrix, which is the weight matrix between the hidder layer and the output layer.
4. $b_2$, a scalar, which is the offset for the second layer.

The following script initializes the above four parameters and returns them as a dictionary.

In [2]:
#Initialize parameters 
num_hidden = 20 #number of neurons in the hidden layer
num_op = 2 #number of neurons in the output layer

def initialize_parameters(size_input, size_hidden, size_output):
    np.random.seed(2)
    W1 = np.random.randn(size_hidden, size_input) * 0.01
    b1 = np.zeros(shape=(size_hidden, 1))
    W2 = np.random.randn(size_output, size_hidden) * 0.01
    b2 = np.zeros(shape=(size_output, 1))
    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2}
    return parameters
parameters = initialize_parameters(X_trn.shape[0], num_hidden, num_op)
print('W1',parameters['W1'].shape)
print('b1',parameters['b1'].shape)
print('W2',parameters['W2'].shape)
print('b2',parameters['b2'].shape)



W1 (20, 196)
b1 (20, 1)
W2 (2, 20)
b2 (2, 1)


## Softmax function.
Let $Z_2=(z_1, z_2)$ be the final otuput layer of neurons. The softmax outputs are probability estimates for outputing label 1 (assuming '8' is 1 and '9' is zero):

$$\hat{y}_1 = Pr(Y = 1 | z_1, z_2) = \frac{e^{z_1}}{e^{z_1} + e^{z_2}}$$

Write code in the cell below to do the softmax computation. Note here Z2 should be a matrix of shape $2 \times n$, where $n$ is the number of training samples. Your output softmax should be of shape $1 \times n$.

In [3]:
import numpy as np

def softmax(Z2):
    # ip - (M,N) array where M is no. of neurons in output layer, N is number of samples.
    # You can modify the code if your output layer is of different dimension
   # =========Write your code below ==============
    n = Z2.shape[1]
    softmax = np.zeros(shape=(1, n))
    def sm(x,y):
        return np.exp(x)/(np.exp(x)+np.exp(y))
    
    smfunc = np.vectorize(sm)
    softmax[0] = smfunc(Z2[0],Z2[1])
    # =============================================
    assert(softmax.shape == (1, Z2.shape[1]))
    return softmax


## Activation function.
The following function should be able to implement activation function given the input.

In [4]:
def activ(ip,act):
    # ip - array obtained after multiplying inputs with weights (between input layer and hidden layer)
    # act - ReLU or Sigmoid
    # I am assuming that "ip" already includes the bias terms, since the bias terms were not separately passed as a parameter
    out = np.zeros(shape=ip.shape)
    
    def m(x):
        return np.maximum(x,0)
    def s(x):
        return 1.0/(1+np.exp(-1*x))
    
    if act =="ReLU":
        # =========Write your code below ==============
        f = np.vectorize(m)
        out = m(ip)

    # =============================================
    elif act == "Sigmoid":
        # =========Write your code below ==============
        f = np.vectorize(s)
        out = f(ip)
                

    # =============================================
    assert(out.shape == ip.shape)
    return out


## <font color='blue'> Definition of "ip" </font> ##
<font color='blue'> I decided to include the bias term inside "ip", as using the global variable b1 outside of the "activ" function is programmatically unsound. Instead, the bias term was added to the product of the input and hidden layer in the forward propagation code below. Functionally, it should be no different and works as expected. This is just a technicallity point for ensuring code readability and debugging.</font>


## Forward Propagation
Given $X, W_1, b_1, W_2, b_2$, the following function will compute the neurons and activated values in the hidden layer, denoted by $Z_1, A_1$ respectively. It will also return the neurons in the last layer and the softmax function computed from it, denoted by $Z_2, A_2$ respectively.

In [5]:
#Forward Propagation   
def forward_propagation(X, parameters, act):
# =========Write your code below ==============
    #Z1 = W1*X+b1
    Z1 = parameters['W1'].dot(X) + parameters['b1']
    A1 = activ(Z1,act)
    Z2 = parameters['W2'].dot(A1) + parameters['b2']
    A2 = softmax(Z2)
    # =============================================
    assert(A2.shape == (1, X.shape[1]))
    
    neuron = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    return neuron



## Backward propagation
In this assignment, we will use the cross-entropy loss defined below as our loss function. Recall that $Z_2=(z_1, z_2)$ is the final layer of neurons, and after softmax we obtain $\hat{y}$ which corresponds to the probability of label 8. Let $y$ be the true labels (assume 1 for '8', 0 for '9')
$$L(y,\hat{y}) = -y\log(\hat{y}) - (1-y)\log(1-\hat{y}),$$
where $$\hat{y} = \frac{e^{z_1}}{e^{z_1} + e^{z_2}}.$$

You have shown in the assignment that:
$$\frac{\partial L(y,\hat{y})}{\partial z_1} = \hat{y} - y, \frac{\partial L(y,\hat{y})}{\partial z_2} = y - \hat{y}.$$
Given the parameters and the neuron values, we can calculate the derivative of the loss function w.r.t all the parameters $W_1, b_1, W_2, b_2$ using backward propagation. Note here, all the gradients should be of the same dimension as the corresponding parameters. 

In [6]:
def backprop(parameters, neuron, X, Y, act):
# =========Write your code below ==============
    # neuron = {"Z1": Z1,
    #         "A1": A1,
    #         "Z2": Z2,
    #         "A2": A2}
    # logistic function
    def sigm(x):
        return 1.0/(1+np.exp(-1*x))
    def sigmaderiv(x):
        return (1.0/(1+np.exp(-1*x)))*(1-1.0/(1+np.exp(-1*x)))
    # heaviside function, derivative of the ReLU activation function
    def heavy(x):
        return np.heaviside(x, 0)
    
    Z1 = neuron["Z1"]
    Z2 = neuron["Z2"]
    A1 = neuron["A1"]
    A2 = neuron["A2"]
    W2 = parameters['W2']
    W1 = parameters['W1']
    dldz2 = np.zeros(shape=Z2.shape)
    
    def dldz2top(x,y):
        return x - y
    def dldz2bottom(x,y):
        return y - x
    topfunc = np.vectorize(dldz2top)
    bottomfunc = np.vectorize(dldz2bottom)
    top = topfunc(A2[0],Y)
    bottom = bottomfunc(A2[0],Y)
    dldz2 = np.vstack((top,bottom))

    # gradients computation
    dW2 = dldz2.dot(A1.transpose())
    db2 = dldz2.dot(np.ones(shape=(X.shape[1],1)))
    # da1dz1 will depend on the activation function
    if act == "Sigmoid":
        da1dz1 = np.zeros(shape=Z1.shape)
        s = np.vectorize(sigmaderiv)
        da1dz1 = s(Z1)
        dW1 = (np.multiply((((dldz2.transpose()).dot(W2)).transpose()),da1dz1)).dot(X.transpose())
        db1 = (np.multiply((((dldz2.transpose()).dot(W2)).transpose()),da1dz1)).dot(np.ones(shape=(X.shape[1],1)))
    if act == "ReLU":
        da1dz1 = np.zeros(shape=Z1.shape)
        g = np.vectorize(heavy)
        da1dz1 = g(Z1)
        dW1 = (np.multiply((((dldz2.transpose()).dot(W2)).transpose()),da1dz1)).dot(X.transpose())
        db1 = (np.multiply((((dldz2.transpose()).dot(W2)).transpose()),da1dz1)).dot(np.ones(shape=(X.shape[1],1)))
    # =============================================
    
    assert(dW1.shape == W1.shape)
    assert(dW2.shape == W2.shape)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

# neuron = forward_propagation(X_trn, parameters, act="Sigmoid")
# print(backprop(parameters, neuron, X_trn, Y_trn, act='Sigmoid')['dW1'])
# print(backprop(parameters, neuron, X_trn, Y_trn, act='Sigmoid')['dW2'])
# print(backprop(parameters, neuron, X_trn, Y_trn, act='Sigmoid')['db1'])
# print(backprop(parameters, neuron, X_trn, Y_trn, act='Sigmoid')['db2'])


def cross_entropy_loss(softmax, Y):
# =========Write your code below ==============
    #L(y,y') = -ylog(y') - (1-y)log(1-y')
    loss = np.zeros(shape=Y.shape)
    def entropy(y,yhat):
        return -y*np.log(yhat) - (1-y)*np.log(1-yhat)
    e = np.vectorize(entropy)
    loss = e(Y,softmax[0])
# =============================================        
    assert(loss.shape == Y.shape)
    return loss


## Parameter updates
Given the parameters and the gradients, we simply update the parameters by the following:

$$W = W - \eta dW$$

where $\eta$ is the learning rate.

In [7]:
def update_parameters(parameters, grads, learning_rate):

# =========Write your code below ==============
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    
    W1 = W1 - learning_rate*grads['dW1']
    W2 = W2 - learning_rate*grads['dW2']
    b1 = b1 - learning_rate*grads['db1']
    b2 = b2 - learning_rate*grads['db2']

# =============================================

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


## Neural network models
Combining the above mentioned parameters, implement the following function to learn a neural network and do inference on it. For prediction, you take the argument that gives the largest softmax output at the last layer.

In [10]:
from sklearn.metrics import accuracy_score
def nn_model1(X_trn, X_tst, Y_trn, Y_tst, n_h, n_o, epochs, act, learning_rate):
    #X_trn: the training set
    #X_tst: the test set
    #Y_trn: training labels
    #Y_tst: test labels
    #n_h: number of neurons in the hidden layer
    #n_o: number of neurons in the output layer
    #epochs: number of epochs for the training
    #act: the activation function you choose
    #learning_rate: a list of length epochs, which consists of the learning rate in each step
    
    def predict(Y, x, parameters, act):
        pred = np.zeros(shape=Y.shape)
        z1 = parameters['W1'].dot(x) + parameters['b1']
        a1 = activ(z1,act)
        z2 = parameters['W2'].dot(a1) + parameters['b2']
        a2 = np.zeros(shape=(1, z2.shape[1]))                  
        for i in range(0,z2.shape[1]):
            a2[0][i] = np.exp(z2[0][i])/(np.exp(z2[0][i])+np.exp(z2[1][i]))
            
        for i in range(0,len(a2[0])):
            if a2[0][i] >= 0.5:
                pred[i] = 1
            else:
                pred[i] = 0
        return pred
    
    assert(len(learning_rate) == epochs)
    err_tst = []
    err_trn = []
    loss_trn = []
    parameters = initialize_parameters(X_trn.shape[0], n_h, n_o)
    
   # =========Write your code below ==============
    for i in range(0,epochs):
        print(i)
        neuron = forward_propagation(X_trn, parameters, act)
        loss = np.sum(cross_entropy_loss(neuron["A2"], Y_trn))/len(Y_trn)
        loss_trn.append(loss)
        grads = backprop(parameters, neuron, X_trn, Y_trn, act)
        parameters = update_parameters(parameters, grads, learning_rate[i])
        trn_pred = predict(Y_trn,X_trn,parameters,act)
        tst_pred = predict(Y_tst,X_tst,parameters,act)
        print("Train error: ")
        print(1-accuracy_score(Y_trn, trn_pred))
        err_trn.append(1-accuracy_score(Y_trn, trn_pred))
        err_tst.append(1-accuracy_score(Y_tst, tst_pred))
        
    # =============================================    
    #err_tst: testing error (classification error) in each epoch
    #err_trn: training error (classification error) in each epoch
    #loss_trn: training loss (cross entropy loss) in each epoch
    #parameters: the final learned parameters
    return err_tst, err_trn, loss_trn, parameters


## Problem 0: Verify that your code is working well.
Using ReLU (Sigmoid) as your activation function, implement a learning algorithm with fixed learning rate $\eta = 0.01$ at each step. Set the number of epochs to be 20000. Plot the cross entropy loss at each epoch to convince yourself that you are training well. (Your cross entropy loss should be decreasing smoothly. This part won't be graded.)

## <font color='blue'> Scaling the Learning Rate </font> ##
<font color='blue'> I added a scaling factor of 1/964 to each of the learning rates in the problems below, as suggested per Ziteng Sun's piazza post.</font>

In [ ]:
epochs = 20000
lr1 = 0.01/964*np.ones(epochs)
# =========Write your code below ==============
err_tst, err_trn, loss_trn, parameters = nn_model1(X_trn, X_tst, Y_trn, Y_tst, 20, 2, epochs, "ReLU", lr1)
# =============================================
plt.figure(1, figsize=(12, 8))
plt.plot(range(epochs), loss_trn, '-', color='orange',linewidth=2, label='training loss (lr = 0.01/964)')
plt.title('Training loss')
plt.xlabel('epoch')
plt.ylabel('Cross entropy error')
plt.legend(loc='best')
plt.grid()
plt.show()

0
Train error: 
0.39419087136929465
1
Train error: 
0.421161825726141
2
Train error: 
0.446058091286307
3
Train error: 
0.4595435684647303
4
Train error: 
0.48755186721991706
5
Train error: 
0.49377593360995853
6
Train error: 
0.49377593360995853
7
Train error: 
0.4927385892116183
8
Train error: 
0.4927385892116183
9
Train error: 
0.4927385892116183
10
Train error: 
0.4927385892116183
11
Train error: 
0.4927385892116183
12
Train error: 
0.4927385892116183
13
Train error: 
0.4927385892116183
14
Train error: 
0.4927385892116183
15
Train error: 
0.4927385892116183
16
Train error: 
0.4927385892116183
17
Train error: 
0.4927385892116183
18
Train error: 
0.4927385892116183
19
Train error: 
0.4927385892116183
20
Train error: 
0.4927385892116183
21
Train error: 
0.4927385892116183
22
Train error: 
0.4927385892116183
23
Train error: 
0.4927385892116183
24
Train error: 
0.4927385892116183
25
Train error: 
0.4927385892116183
26
Train error: 
0.4927385892116183
27
Train error: 
0.4927385892116183


Train error: 
0.4927385892116183
228
Train error: 
0.4927385892116183
229
Train error: 
0.4927385892116183
230
Train error: 
0.4927385892116183
231
Train error: 
0.4927385892116183
232
Train error: 
0.4927385892116183
233
Train error: 
0.4927385892116183
234
Train error: 
0.4927385892116183
235
Train error: 
0.4927385892116183
236
Train error: 
0.4927385892116183
237
Train error: 
0.4927385892116183
238
Train error: 
0.4927385892116183
239
Train error: 
0.4927385892116183
240
Train error: 
0.4927385892116183
241
Train error: 
0.4927385892116183
242
Train error: 
0.4927385892116183
243
Train error: 
0.4927385892116183
244
Train error: 
0.4927385892116183
245
Train error: 
0.4927385892116183
246
Train error: 
0.4927385892116183
247
Train error: 
0.4927385892116183
248
Train error: 
0.4927385892116183
249
Train error: 
0.4927385892116183
250
Train error: 
0.4927385892116183
251
Train error: 
0.4927385892116183
252
Train error: 
0.4927385892116183
253
Train error: 
0.4927385892116183
254
T

Train error: 
0.4927385892116183
452
Train error: 
0.4927385892116183
453
Train error: 
0.4927385892116183
454
Train error: 
0.4927385892116183
455
Train error: 
0.4927385892116183
456
Train error: 
0.4927385892116183
457
Train error: 
0.4927385892116183
458
Train error: 
0.4927385892116183
459
Train error: 
0.4927385892116183
460
Train error: 
0.4927385892116183
461
Train error: 
0.4927385892116183
462
Train error: 
0.4927385892116183
463
Train error: 
0.4927385892116183
464
Train error: 
0.4927385892116183
465
Train error: 
0.4927385892116183
466
Train error: 
0.4927385892116183
467
Train error: 
0.4927385892116183
468
Train error: 
0.4927385892116183
469
Train error: 
0.4927385892116183
470
Train error: 
0.4927385892116183
471
Train error: 
0.4927385892116183
472
Train error: 
0.4927385892116183
473
Train error: 
0.4927385892116183
474
Train error: 
0.4927385892116183
475
Train error: 
0.4927385892116183
476
Train error: 
0.4927385892116183
477
Train error: 
0.4927385892116183
478
T

Train error: 
0.4927385892116183
675
Train error: 
0.4927385892116183
676
Train error: 
0.4927385892116183
677
Train error: 
0.4927385892116183
678
Train error: 
0.4927385892116183
679
Train error: 
0.4927385892116183
680
Train error: 
0.4927385892116183
681
Train error: 
0.4927385892116183
682
Train error: 
0.4927385892116183
683
Train error: 
0.4927385892116183
684
Train error: 
0.4927385892116183
685
Train error: 
0.4927385892116183
686
Train error: 
0.4927385892116183
687
Train error: 
0.4927385892116183
688
Train error: 
0.4927385892116183
689
Train error: 
0.4927385892116183
690
Train error: 
0.4927385892116183
691
Train error: 
0.4927385892116183
692
Train error: 
0.4927385892116183
693
Train error: 
0.4927385892116183
694
Train error: 
0.4927385892116183
695
Train error: 
0.4927385892116183
696
Train error: 
0.4927385892116183
697
Train error: 
0.4927385892116183
698
Train error: 
0.4927385892116183
699
Train error: 
0.4927385892116183
700
Train error: 
0.4927385892116183
701
T

Train error: 
0.4927385892116183
899
Train error: 
0.4927385892116183
900
Train error: 
0.4927385892116183
901
Train error: 
0.4927385892116183
902
Train error: 
0.4927385892116183
903
Train error: 
0.4927385892116183
904
Train error: 
0.4927385892116183
905
Train error: 
0.4927385892116183
906
Train error: 
0.4927385892116183
907
Train error: 
0.4927385892116183
908
Train error: 
0.4927385892116183
909
Train error: 
0.4927385892116183
910
Train error: 
0.4927385892116183
911
Train error: 
0.4927385892116183
912
Train error: 
0.4927385892116183
913
Train error: 
0.4927385892116183
914
Train error: 
0.4927385892116183
915
Train error: 
0.4927385892116183
916
Train error: 
0.4927385892116183
917
Train error: 
0.4927385892116183
918


# Problem 1: Learning with fixed learning rate.
Using ReLU as your activation function, implement a learning algorithm with fixed learning rate $\eta = 0.01$ at each step. Plot the training and testing error (classification error) you get at each epoch. Justify your plot. (Set the number of hidden neurons in the hidden layer to be 20 for problem 1-3, for all problems below, set epochs = 20000).

In [ ]:
epochs = 20000
lr1 = 0.01/964*np.ones(epochs)
# =========Write your code below ==============
err_tst, err_trn, loss_trn, parameters = nn_model1(X_trn, X_tst, Y_trn, Y_tst, 20, 2, epochs, "ReLU", lr1)

# =============================================
plt.figure(1, figsize=(12, 8))
plt.plot(range(epochs), err_trn, '-', color='orange',linewidth=2, label='training error (lr = 0.01/964)')
plt.plot(range(epochs), err_tst, '-b', linewidth=2, label='test error (lr = 0.01/964)')


plt.title('ReLU(Learning rate=0.01/964)')
plt.xlabel('epoch')
plt.ylabel('error')
plt.legend(loc='best')
plt.grid()
plt.show()

## <font color='blue'> Explanation for Problem 1 </font> ##
<font color='blue'> At a learning rate of $\eta = 0.01/984$ with the ReLU activation function, the training error and testing error are decreasing very slowly for the first 2400 epochs. The errors for both training and testing hover around 50%, with the training error slightly lower (around 49/48%). Beyond 2500 epochs, we see significant reduction in the training and testing errors from 50% to 17% at 5000 epochs. Beyond 10000 epochs, the testing and training errors are convering to a value of 17% and 13% respectively. At the end, the training error reaches a minimum of 12%, resulting in a overall end training accuracy of 88%. The overall end testing accuracy is around 82%. As expected, the training error is lower than the testing error, as the model is expected to perform on data it has already seen.</font>

# Problem 2: 
Using ReLU as your activation function, change the learning rate to $\eta = 0.1$. Plot the plots on the same figure as in problem 1. Compare the plots and justify.

In [ ]:
epochs = 20000
lr2 = 0.1/964*np.ones(epochs)
# =========Write your code below ==============
err_tst2, err_trn2, loss_trn2, parameters2 = nn_model1(X_trn, X_tst, Y_trn, Y_tst, 20, 2, epochs, "ReLU", lr2)
# =============================================
plt.figure(2, figsize=(12, 8))
# Classification errors for learning rate = 0.01/964, Relu Activation, n_hidden = 20
plt.plot(range(epochs), err_trn, '-', color='orange',linewidth=2, label='training error (lr = 0.01/964)')
plt.plot(range(epochs), err_tst, '-b', linewidth=2, label='test error (lr = 0.01/964)')

# Classification errors for learning rate = 0.1/964, Relu Activation, n_hidden = 20
plt.plot(range(epochs), err_trn2, '-', linewidth=2, label='training error (lr = 0.1/964)')
plt.plot(range(epochs), err_tst2, '-b', color='yellow', linewidth=2,  label='test error (lr = 0.1/964)')

plt.title('ReLU(Learning rate=0.1/964)')
plt.xlabel('epoch')
plt.ylabel('error')
plt.legend(loc='best')
plt.grid()
plt.show()

## <font color='blue'> Explanation for Problem 2 </font> ##
<font color='blue'> At a learning rate of $\eta = 0.1/984$ with the ReLU activation function, the training error and testing error are decreasing very quickly for the first 1000 epochs. Beyond 200 epochs, we see significant reduction in the training and testing errors from 50% to 15% at 1000 epochs. Beyond 1000 epochs, the testing errors are convering to a value of 17%, before rising back up to around 21%. The training error continues to decline steadily, converging to an error of around 13% between epochs 2500 and 10000. Beyond 10000 epochs, the training error continues to decline to a final training error of 5%. This may be an indication of overfitting, as the training error is significantly lower than the testing error, indicating that the model has "memorized" the training data. Comparing this learning rate of 0.1/984 to a learning rate of 0.01/984, we see that the learning rate of 0.01/984 achieves better testing accuracy, while a learning rate of 0.1/984 achieves a better training accuracy.</font>

# Problem 3: Learning with variable learning rate.
Using ReLU as your activation function, implement a learning algorithm with variable learning rate $\eta = \frac1{\sqrt{i+1}}$ at the $i$th step. Plot the training and testing error you get at each iteration and compare it with the plots you get previously. Justify your plot.

In [ ]:
indices = np.array(range(epochs))
lr3 = 1/np.sqrt(indices + 1)*(1.0/964)
# =========Write your code below ==============
err_tst3, err_trn3, loss_trn3, parameters3 = nn_model1(X_trn, X_tst, Y_trn, Y_tst, 20, 2, epochs, "ReLU", lr3)
# =============================================
plt.figure(3, figsize=(12, 8))
# Classification errors for learning rate = 0.01/964, Relu Activation, n_hidden = 20
plt.plot(range(epochs), err_trn, '-', color='orange',linewidth=2, label='training error (lr = 0.01/964)')
plt.plot(range(epochs), err_tst, '-b', linewidth=2, label='test error (lr = 0.01/964)')

# Classification errors for learning rate = 0.1/964, Relu Activation, n_hidden = 20
plt.plot(range(epochs), err_trn2, '-', color='red', linewidth=2, label='training error (lr = 0.1/964)')
plt.plot(range(epochs), err_tst2, '-b', color='yellow', linewidth=2, label='test error (lr = 0.1/964)')

# Classification errors for variable learning rate, Relu Activation, n_hidden = 20
plt.plot(range(epochs), err_trn3, '-', color='purple', linewidth=2, label='training error (unfixed lr)')
plt.plot(range(epochs), err_tst3, '-b', color='green', linewidth=2, label='test error (unfixed lr)')
plt.title('ReLU(Variable learning rate)')
plt.xlabel('epoch')
plt.ylabel('error')
plt.legend(loc='best')
plt.grid()
plt.show()

## <font color='blue'> Explanation for Problem 3 </font> ##
<font color='blue'> Using a variable learning rate with the ReLU activation function, both the training and testing error plots seem superior than the previous two configurations. The variable learning rate configuration converges to a training error of 13% beyond 5000 epochs and does not vary too much beyond that point. The testing error also converges to an error of 17% beyond 5000 epochs and is almost completely flat beyond that. The final testing error mimics that of the $\eta = 0.01/984$ configuration. The use of adaptive learning rate allows the rate to be quite high initially and then decrease once the model starts to converge, which helps the model learn the correct weights optimally.</font>

# Problem 4: Larger hidden layer.
Change the number of neurons in the hidden layer to be $50$. Redo the experiment in problem 1. Plot all four plots in the same figure and justify your plot.

In [ ]:
num_hidden2 = 50
# =========Write your code below ==============
err_tst4, err_trn4, loss_trn4, parameters4 = nn_model1(X_trn, X_tst, Y_trn, Y_tst, num_hidden2, 2, epochs, "ReLU", lr1)

# =============================================
plt.figure(4, figsize=(12, 8))
# Classification errors for learning rate = 0.01/964, Relu Activation, n_hidden = 20
plt.plot(range(epochs), err_trn, '-', color='orange',linewidth=2, label='training error (#hidden = 20)')
plt.plot(range(epochs), err_tst, '-b', linewidth=2, label='test error (#hidden = 20)')

# Classification errors for learning rate = 0.01/964, Relu Activation, n_hidden = 50
plt.plot(range(epochs), err_trn4, '-', color='red', linewidth=2, label='training error (#hidden = 50)')
plt.plot(range(epochs), err_tst4, '-b', color='grey', linewidth=2, label='test error (#hidden = 50)')

plt.title('ReLU(Learning rate=0.01/964)')
plt.xlabel('epoch')
plt.ylabel('error')
plt.legend(loc='best')
plt.grid()
plt.show()

## <font color='blue'> Explanation for Problem 4 </font> ##
<font color='blue'> By using 50 neurons in the hidden layer instead of 20, we see a slight improvement in the training accuracy, however not by too much. For the testing accuracy, the behavior is almost identical to that of the configuration with 20 neurons. Thus, the addition of more neurons, thereby increasing computationally complexity, does not seem to improve performance in this case by too much.</font>

# Problem 5: Sigmoid Activation.
Change the activation function to be Sigmoid function. Redo the experiment in problem 1. Plot all four plots in the same figure and justify your plot.

In [ ]:
# =========Write your code below ==============
epochs = 20000
lr1 = 0.01/964*np.ones(epochs)
err_tst5, err_trn5, loss_trn5, parameters5 = nn_model1(X_trn, X_tst, Y_trn, Y_tst, 20, 2, epochs, "Sigmoid", lr1)

# =============================================
# Classification errors for learning rate = 0.01/964, Relu Activation, n_hidden = 20
plt.figure(5, figsize=(12, 8))
plt.plot(range(epochs), err_trn, '-', color='orange',linewidth=2, label='training error (ReLU)')
plt.plot(range(epochs), err_tst, '-b', linewidth=2, label='test error (ReLU)')

# Classification errors for learning rate = 0.01/964, Sigmoid Activation, n_hidden = 20
plt.plot(range(epochs), err_trn5, '-', color='red',  linewidth=2, label='training error (Sigmoid)')
plt.plot(range(epochs), err_tst5, '-b', color='green', linewidth=2, label='test error (Sigmoid)')

plt.title('Learning rate=0.01/964')
plt.xlabel('epoch')
plt.ylabel('error')
plt.legend(loc='best')
plt.grid()
plt.show()

## <font color='blue'> Explanation for Problem 5 </font> ##
<font color='blue'> By using a sigmoid activation function instead of a ReLU activation function, the model seems to learn extremely slow. In the first 10000 epochs, the model with Sigmoid activation does not see much change in training and testing error at all. Beyond 10000 epochs, the training and testing errors starts to decrease to around 18% at an epoch of 15000. Beyond 15000 epochs, the model starts to converge. Comparing this with the ReLU model, we see that the ReLU model shows better performance and faster convergence on the training set (lower training error) and faster convergence on the testing set. At the end, near 17500 epochs, the testing error of the sigmoid model is actally slightly lower than the ReLU model. Overall, the ReLU activation function is superior, as it is more computationally efficient and demonstrates better convergence behavior. The Sigmoid model takes much longer to compute and does not converge until a very high epoch count.</font>